### SQL Databases

In [2]:
### Creating sample SQLite Database

import sqlite3
import os

os.makedirs('data/databases', exist_ok=True)

In [ ]:
### Create a sample database

conn = sqlite3.connect('data/databases/company.db')
cursor = conn.cursor()


In [ ]:
#Create tables

cursor.execute('''CREATE TABLE IF NOT EXISTS employee
               (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')


In [5]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
               (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [ ]:
# Insert sample data
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]


In [10]:
cursor.executemany('INSERT OR REPLACE INTO employee VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [13]:
conn.commit()
conn.close()

### Database Content Extraction 

In [15]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [20]:
# Method1 SQLDATABASE Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

###get Database info
print(f"Table: {db.get_usable_table_names()}")
print(f"\n Table DDL:")
print(db.get_table_info())

Table: ['employee', 'projects']

 Table DDL:

CREATE TABLE employee (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employee table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


In [28]:
from typing import List
from langchain_core.documents import Document
###Method 2: Custom SQL to Document conversion

print("\n Custome SQL Processing")

def sql_to_documents(db_path:str)->List[Document]:
    """Convert SQL database to documents with context"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents=[]

    #Strategy 1: Create documents for each table 
    cursor.execute("SELECT name from sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]

        #Get table schema 
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]

        ##Get table data
        cursor.execute(f"SELECT * from {table_name}")
        rows = cursor.fetchall()

        #Create table overview document
        table_content = f"Table :{table_name}\n"
        table_content += f"Columns: {' '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"

        #Add sample records
        table_content += "Sample Records:\n"
        for row in rows[:5]: #First 5 records
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"

        doc = Document(
            page_content=table_content,
            metadata = {
                'source':db_path,
                'table_name':table_name,
                'num_records':len(rows),
                'data_type':'sql_table'
            }
        )
        documents.append(doc)

    ###Strategy 2: Create relationship documents 
    #Example : Join employees and projects
    cursor.execute("""SELECT e.name, e.role, p.name as project_name, p.status
                   FROM employee e
                   JOIN projects p ON e.id=p.lead_id""")
    relationships = cursor.fetchall()
    rel_content = "Employee_Project Relationships:\n\n"
    for rel in relationships:
        rel_content += f"{rel[0]} ({rel[1]}) leads {rel[2]} Status: {rel[3]}\n"

    rel_doc = Document(
        page_content=rel_content, 
        metadata = {
            'source':db_path, 
            'data_type':'sql_relationships',
            'query':'employee_project_join'
        }
    ) 
    documents.append(rel_doc)

    conn.close()
    return documents



 Custome SQL Processing


In [40]:
sql_data = sql_to_documents("data/databases/company.db")
print(sql_data[0].page_content)
print("-"*150)
print(sql_data[1].page_content)
print("-"*150)
print(sql_data[2].page_content)

Table :employee
Columns: id name role department salary
Total Records: 4

Sample Records:
{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}
{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}
{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}
{'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}

------------------------------------------------------------------------------------------------------------------------------------------------------
Table :projects
Columns: id name status budget lead_id
Total Records: 4

Sample Records:
{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.0, 'lead_id': 1}
{'id': 2, 'name': 'Data Pipeline', 'status': 'Completed', 'budget': 80000.0, 'lead_id': 2}
{'id': 3, 'name': 'Customer Portal', 'status': 'Planning